In [166]:
import cv2
import numpy as np
import onnxruntime as ort

In [167]:
model_path = r"D:\DS\ViolenceDetection\notebooks\model_quantized.onnx"

In [168]:
video_path=r'D:\DS\ViolenceDetection\notebooks\violentVideo\violentVideo\vid_1.mp4'

In [169]:
def img_preprocessing(img):
    mean=[0.485, 0.456, 0.406]
    std=[0.229, 0.224, 0.225]
    img = cv2.resize(img, (256, 256))
    img = cv2.cvtColor(img, cv2.COLOR_BGR2RGB)
    img = img / 255.0
    for i in range(3):
        img[:, :, i] = (img[:, :, i] - mean[i]) / std[i]
    img = np.transpose(img, (2, 0, 1))
    return img

In [170]:
def predict_on_video(video_path,onnx_model_path,n_fps_pred=7,n_frames_pred=8,n_frames_split=1):
    #loader model
    
    session = ort.InferenceSession(onnx_model_path)
    input_name=session.get_inputs()[0].name
    output_name=session.get_outputs()[0].name

    cap = cv2.VideoCapture(video_path)
    if not cap.isOpened():
        print("Error: Could not open video.")
        return
    fps=cap.get(cv2.CAP_PROP_FPS)
    interval=int(fps/n_fps_pred)
    X=[]
    violent_frames=[]
    tmp_violent_frames=[]
    counts=0
    while True:
        counts+=1
        ret,frame=cap.read()
        if not ret:
            break
        if(counts%interval==0):
            counts=0
            processed_frame=img_preprocessing(frame)
            X.append(processed_frame)
            tmp_violent_frames.append(frame)
        input_data=np.array([X.copy()],dtype=np.float32)
        if(len(input_data[0])!=n_frames_pred):
            pass
        else:
            outputs = session.run([output_name], {input_name: input_data})
            y_preds=np.argmax(outputs[0],axis=1)[0]
            print(outputs[0][0])
            if(y_preds==1):
                if(len(violent_frames)>=n_frames_pred):
                    violent_frames.extend(tmp_violent_frames[-(n_frames_pred-int(n_frames_pred/n_frames_split)+1):])
                else:
                    violent_frames.extend(tmp_violent_frames)
                print(y_preds)
            X=X[-int(n_frames_pred/n_frames_split)+1:]
            tmp_violent_frames=tmp_violent_frames[-int(n_frames_pred/n_frames_split)+1:]
        cv2.imshow('Video',frame)
        if cv2.waitKey(1) & 0xFF == ord('q'):
                break
        if not ret:
                break
    cap.release()
    cv2.destroyAllWindows()

    return violent_frames
        

In [171]:
violentFrames=predict_on_video(video_path,model_path)

[ 0.22819023 -0.42857793]
[ 0.19774565 -0.3949933 ]
[ 0.3837733  -0.59285223]
[ 0.21789409 -0.41462424]
[ 0.1346019 -0.3211243]
[ 0.17677037 -0.36847928]
[ 0.14300424 -0.33060318]
[ 0.37218156 -0.580194  ]
[ 0.3009653  -0.50478727]
[ 0.30545017 -0.510944  ]
[ 0.40407506 -0.6182516 ]
[ 0.4336272 -0.6455287]
[ 0.39591718 -0.60481495]
[ 0.6886935  -0.90367264]
[ 1.0908   -1.301117]
[ 0.98011506 -1.1866554 ]
[ 0.6559317  -0.85423374]
[ 0.6560701  -0.85592705]
[ 0.6326157  -0.83815676]
[ 0.73373073 -0.94680697]
[ 0.77509844 -0.9873491 ]
[ 0.77073514 -0.984622  ]
[ 0.69331753 -0.90898544]
[ 0.5231742  -0.73712325]
[ 0.5375141  -0.74767053]
[ 0.55069613 -0.759185  ]
[ 0.566444  -0.7758583]
[ 0.5671398 -0.7798783]
[ 0.41543105 -0.6226952 ]
[ 0.26590493 -0.4677727 ]
[ 0.26206124 -0.4662436 ]
[ 0.21948174 -0.4195002 ]
[ 0.22827452 -0.4293356 ]
[ 0.2307041  -0.43104553]
[ 0.1754841 -0.3686904]
[ 0.12818097 -0.31328017]
[ 0.12351117 -0.30860373]
[ 0.14968146 -0.33950981]
[ 0.2070544  -0.40676862]


In [172]:
for i in range(len(violentFrames)):
    # Show each frame
    # if(i>=12):
    try:

        cv2.imshow('Video Frame', violentFrames[i])
    except:
        print(violentFrames[i])
        break
    # else:
    #     cv2.imshow('Video Frame', violentFrames[i])
    
    # Wait for 25 ms between frames (40 FPS). Adjust as needed.
    if cv2.waitKey(100) & 0xFF == ord('q'):
        break  # Press 'q' to exit early

cv2.destroyAllWindows()

In [173]:
violentFrames

[array([[[114, 143, 139],
         [114, 143, 139],
         [114, 143, 139],
         ...,
         [207, 236, 232],
         [207, 236, 232],
         [207, 236, 232]],
 
        [[114, 143, 139],
         [114, 143, 139],
         [114, 143, 139],
         ...,
         [207, 236, 232],
         [207, 236, 232],
         [207, 236, 232]],
 
        [[114, 143, 139],
         [114, 143, 139],
         [114, 143, 139],
         ...,
         [209, 238, 234],
         [209, 238, 234],
         [209, 238, 234]],
 
        ...,
 
        [[132, 137, 143],
         [135, 140, 146],
         [131, 139, 143],
         ...,
         [142, 125, 128],
         [142, 125, 128],
         [142, 125, 128]],
 
        [[137, 142, 148],
         [137, 142, 148],
         [136, 141, 147],
         ...,
         [142, 125, 128],
         [142, 125, 128],
         [142, 125, 128]],
 
        [[139, 144, 150],
         [139, 144, 150],
         [142, 147, 153],
         ...,
         [142, 125, 128],
  